In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
def tokenize(word):
    return nltk.word_tokenize(word)

In [3]:
tokenize('how are you ? bro ')

['how', 'are', 'you', '?', 'bro']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('how'),stem('are'),stem('you'),stem('?'),stem('bro')

('how', 'are', 'you', '?', 'bro')

In [6]:
def bag_of_words(tokenized_word,all_words):
    tokenized_words = []
    for tokenized_w in tokenized_word:
        tokenized_words.append(stem(tokenized_w))
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(tokenized_words):
        if w in all_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['bro'],tokenize('how are you ? bro '))

array([1., 0., 0., 0., 0.], dtype=float32)

In [8]:
batch_size = 8
epochs = 1000
device = 'cuda'
new_data = []
all_words = []
tags = []
ignore_words = ["?", "!", ".", "@", "#", "$", "%", "^", "&", "*"]
data = pd.read_csv('./data/train_data_cleaning.csv')
X = data['text'].tolist()
y = data['target'].tolist()

In [9]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for x_batch in X_batch:
        if x_batch in ignore_words:
            pass
        else:
            new_X.append(stem(x_batch))
        all_words.extend(new_X)
    tags.append(y_batch)
    new_data.append((new_X,y_batch))

7613it [00:02, 3079.72it/s]


In [10]:
tags = sorted(set(tags))

In [11]:
tags

[0, 1]

In [12]:
all_words = sorted(set(all_words))

In [13]:
np.random.shuffle(new_data)

In [14]:
X = []
y = []

In [15]:
for sent,tag in tqdm(new_data):
    bag = bag_of_words(sent,all_words)
    X.append(bag)
    label = tags.index(tag)
    y.append(label)

100%|██████████| 7613/7613 [00:12<00:00, 630.42it/s]


In [16]:
X[:5]

[array([1., 1., 1., ..., 0., 0., 0.], dtype=float32),
 array([1., 1., 1., ..., 0., 0., 0.], dtype=float32),
 array([1., 1., 1., ..., 0., 0., 0.], dtype=float32),
 array([1., 1., 1., ..., 0., 0., 0.], dtype=float32),
 array([1., 1., 1., ..., 0., 0., 0.], dtype=float32)]

In [17]:
y[:5]

[0, 1, 1, 0, 1]

In [18]:
len(all_words)

13918

In [19]:
from sklearn.model_selection import *

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)

In [21]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [22]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds.view(-1,1),y.view(-1,1))
    return loss.item()

In [23]:
def get_accuracy(model,X,y):
    preds = model(X)
    correct = -1
    total = -1
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        y_batch = int(torch.round(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)
    acc = acc*100
    return acc

In [ ]:
class Model(Module):
    def __init__(self,hidden=128):
        super().__init__()
        self.activation = ReLU()
        self.linear1 = Linear(len(all_words),hidden)
        self.linear2 = Linear(hidden,hidden*2)
        self.linear3 = Linear(hidden*2,hidden*4)
        self.linear4 = Linear(hidden*4,hidden*2)
        self.linear5 = Linear(hidden*2,1)
        self.output_activation = Sigmoid()
    
    def forward(self,X):
        preds = self.activation(self.linear1(X))
        preds = self.activation(self.linear2(preds))
        preds = self.activation(self.linear3(preds))
        preds = self.activation(self.linear4(preds))
        preds = self.output_activation(self.linear5(preds))
        return preds

In [ ]:
PROJECT_NAME = 'NLP-with-Disaster-Tweets-Clf'

In [ ]:
import wandb

In [ ]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
wandb.init(project=PROJECT_NAME,name=f'baseline-{criterion}-backward-pass')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        preds = preds.to(device)
        loss = criterion(preds.view(-1,1),y_batch.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
wandb.watch(model)
wandb.finish()

wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


  1%|▏         | 13/1000 [00:26<32:05,  1.95s/it]

In [31]:
torch.round(preds)

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.]], device='cuda:0', grad_fn=<RoundBackward>)

In [32]:
y_batch

tensor([1., 0., 1., 0., 1.], device='cuda:0')

In [33]:
X_batch

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]], device='cuda:0')